# Load libraries and functions

In [98]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [99]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

## Main Code

In [100]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# # occupation = 'shampooers'
# # occupation = 'dredgeOperators'


GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [101]:
# Load the data
onet = pd.read_csv(f'{data_path}/data/onet_occupations_yearly.csv')
onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
onet = onet[onet['year'] == 2023].reset_index(drop=True)

# Get list of tasks
my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
tasks = my_df['task'].unique().tolist()
tasks

['Decline excessive risks.',
 'Write to field representatives, medical personnel, or others to obtain further information, quote rates, or explain company underwriting policies.',
 'Evaluate possibility of losses due to catastrophe or excessive insurance.',
 'Decrease value of policy when risk is substandard and specify applicable endorsements or apply rating to ensure safe, profitable distribution of risks, using reference materials.',
 'Review company records to determine amount of insurance in force on single risk or group of closely related risks.',
 'Authorize reinsurance of policy when risk is high.',
 'Examine documents to determine degree of risk from factors such as applicant health, financial standing and value, and condition of property.']

<br>

<br>

# 2) GPT Naive DAG

In [102]:
version = 'v0/'
version = 'v1/'

In [103]:
# Set output names
output_filename_wo = f'{occupation_folder}/{version}{occupation}_oneStepGPT_DAG_df.csv'
output_filename_w = f'{occupation_folder}/{version}{occupation}_twoStepGPT_DAG_df.csv'

### Set up the questions for GPT

In [104]:
if version == 'v0/':
    task_relationships_question_options_wo = {'A first': "A would be done first",
                                            'B first': "B would be done first",
                                            'neither': "These are not part of the same task sequence"
                                            }

    task_relationships_question_options_w = {'A first': "A would be done first",
                                            'B first': "B would be done first",
                                            'either': "Could be done in either order, but still part of the same sequence",
                                            'neither': "These are not part of the same task sequence"
                                            }

    symmetric_edges_question_options = {'A first': "A would be done first",
                                        'B first': "B would be done first",
                                        }

    task_relationships_question_text = dedent("""\
                                            Consider {{ occupation }} as an occupation. 
                                            And consider these two tasks: 
                                            A) {{ task_A }} 
                                            B) {{ task_B }}
                                            What is the relationship between these tasks?
                                            Avoid using words like "task A" and "task B" in the answer.
                                            Explain the reasoning behind your answer in a couple of sentences.
                                            """)

    symmetric_edges_question_text = dedent("""\
                                        Consider {{ occupation }} as an occupation. 
                                        And consider these two tasks: 
                                        A) {{ task_A }} 
                                        B) {{ task_B }}
                                        What is the relationship between these tasks?
                                        Avoid using words like "task A" and "task B" in the answer.
                                        Explain the reasoning behind your answer in a couple of sentences.
                                        """)

In [105]:
if version == 'v1/':
  task_relationships_question_options_wo = {'A first': "Worker working on task B needs to know the output of worker working on task A",
                                            'B first': "Worker working on task A needs to know the output of worker working on task B",
                                            'neither': "Neither worker needs to know the output of the other worker"
                                            }

  task_relationships_question_options_w = {'A first': "Worker working on task B needs to know the output of worker working on task A",
                                            'B first': "Worker working on task A needs to know the output of worker working on task B",
                                            'either': "Either worker can start first, but the output of one worker is needed by the other worker",
                                            'neither': "Neither worker needs to know the output of the other worker"
                                          }

  symmetric_edges_question_options = {'A first': "Worker working on task B needs to know the output of worker working on task A",
                                      'B first': "Worker working on task A needs to know the output of worker working on task B",
                                      }

  task_relationships_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation. 
                                          And consider these two tasks: 
                                          A) {{ task_A }} 
                                          B) {{ task_B }}
                                          Imagine there are two workers, one working on task A and the other on task B.
                                          Does the worker working on task B need to know the output of the worker working on task A before getting started? What about the opposite?
                                          Avoid using words like "task A" and "task B" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)

  symmetric_edges_question_text = dedent("""\
                                        Consider {{ occupation }} as an occupation. 
                                        And consider these two tasks: 
                                        A) {{ task_A }} 
                                        B) {{ task_B }}
                                        Imagine there are two workers, one working on task A and the other on task B.
                                        Does the worker working on task B need to know the output of the worker working on task A before getting started? What about the opposite?
                                        Avoid using words like "task A" and "task B" in the answer.
                                        Explain the reasoning behind your answer in a couple of sentences.
                                        """)

In [106]:
task_relationships_question_options_wo_list = list(task_relationships_question_options_wo.values())
task_relationships_question_options_w_list = list(task_relationships_question_options_w.values())
symmetric_edges_question_options_list = list(symmetric_edges_question_options.values())

### 2.1) One Step Method: Directly ask for pairwise comparison w/o giving the "either" option

In [107]:
# Compare pair of tasks
def task_relationships(occupation, tasks, question_text, question_options):
    if '"Sink"' in tasks:
        tasks.remove('"Sink"')

    scenarios = [Scenario({"occupation": occupation, "task_A": task_A, "task_B": task_B}) 
        for task_A, task_B in combinations(tasks, 2)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results

results = task_relationships(GPT_input_occupation, tasks, task_relationships_question_text, task_relationships_question_options_wo_list)
#results.select("task_A", "task_B", "ordering", "comment.ordering_comment").print()
pairwise_relationships_wo_raw = results.select("task_A", "task_B", "ordering", "comment.ordering_comment").to_pandas()

Output()

In [108]:
# Swap columns and subset only those that are part of the same task sequence 
pairwise_relationships_wo = pairwise_relationships_wo_raw.copy()
mask = pairwise_relationships_wo['answer.ordering'] == task_relationships_question_options_wo['B first']
pairwise_relationships_wo.loc[mask, ['scenario.task_A', 'scenario.task_B']] = pairwise_relationships_wo.loc[mask, ['scenario.task_B', 'scenario.task_A']].values
pairwise_relationships_wo.loc[mask, 'answer.ordering'] = task_relationships_question_options_wo['A first']
pairwise_relationships_wo = pairwise_relationships_wo[pairwise_relationships_wo['answer.ordering'] == task_relationships_question_options_wo['A first']]
pairwise_relationships_wo = pairwise_relationships_wo[['scenario.task_A', 'scenario.task_B', 'comment.ordering_comment']]

# Change column names
pairwise_relationships_wo = pairwise_relationships_wo.rename(columns={'scenario.task_A': 'source', 
                                                                      'scenario.task_B': 'target', 
                                                                      'comment.ordering_comment': 'comment'})

# Save output
pairwise_relationships_wo.to_csv(output_filename_wo, index=False)

### 2.2) Two Steps Method: Give option of "either" and then filter symmetric edges
### Step 1:

In [109]:
# Compare pair of tasks
def task_relationships(occupation, tasks, question_text, question_options):
    if '"Sink"' in tasks:
        tasks.remove('"Sink"')

    scenarios = [Scenario({"occupation": occupation, "task_A": task_A, "task_B": task_B}) 
        for task_A, task_B in combinations(tasks, 2)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results

results = task_relationships(GPT_input_occupation, tasks, task_relationships_question_text, task_relationships_question_options_w_list)
#results.select("task_A", "task_B", "ordering", "comment.ordering_comment").print()
pairwise_relationships_w_raw = results.select("task_A", "task_B", "ordering", "comment.ordering_comment").to_pandas()

Output()

### Step 2:

In [110]:
# subset symmetric edges
both_edges = pairwise_relationships_w_raw[pairwise_relationships_w_raw['answer.ordering'] == task_relationships_question_options_w['either']]
task_A_list = both_edges['scenario.task_A'].tolist()
task_B_list = both_edges['scenario.task_B'].tolist()


# Decide which one of symmetric edges to keep
def pick_oneOf_symmetricEdges(occupation, task_A_list, task_B_list, question_text, question_options):
    scenarios = [Scenario({"occupation": occupation, "task_A": task_A, "task_B": task_B}) 
        for task_A, task_B in zip(task_A_list, task_B_list)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results

results = pick_oneOf_symmetricEdges(GPT_input_occupation, task_A_list, task_B_list, task_relationships_question_text, symmetric_edges_question_options_list)
#results.select("task_A", "task_B", "ordering", "comment.ordering_comment").print()
which_symmetric_edge = results.select("task_A", "task_B", "ordering", "comment.ordering_comment").to_pandas()

IndexError: list index out of range

In [ ]:
# Merge datasets
pairwise_relationships_w = pairwise_relationships_w_raw[pairwise_relationships_w_raw['answer.ordering'].isin(symmetric_edges_question_options_list)]
pairwise_relationships_w = pd.concat([pairwise_relationships_w, which_symmetric_edge], ignore_index=True)

# Swap columns
mask = pairwise_relationships_w['answer.ordering'] == task_relationships_question_options_w['B first']
pairwise_relationships_w.loc[mask, ['scenario.task_A', 'scenario.task_B']] = pairwise_relationships_w.loc[mask, ['scenario.task_B', 'scenario.task_A']].values
pairwise_relationships_w.loc[mask, 'answer.ordering'] = task_relationships_question_options_w['A first']
pairwise_relationships_w = pairwise_relationships_w[pairwise_relationships_w['answer.ordering'] == task_relationships_question_options_w['A first']]
pairwise_relationships_w = pairwise_relationships_w[['scenario.task_A', 'scenario.task_B', 'comment.ordering_comment']]

# Change column names
pairwise_relationships_w = pairwise_relationships_w.rename(columns={'scenario.task_A': 'source', 
                                                                    'scenario.task_B': 'target', 
                                                                    'comment.ordering_comment': 'comment'})

# Save output
pairwise_relationships_w.to_csv(output_filename_w, index=False)